# 🏋️ Workout Data Natural Language Query Interface

This notebook allows you to interactively test natural language queries against your Concept2 workout database using the MCP server.

## What You Can Do:
- Ask questions in plain English about your workouts
- See how natural language gets parsed into structured queries
- View the dynamically generated SQL
- Get results from your workout database via MCP server

## Example Questions:
- "What was my longest workout?"
- "What's my best 6k time?"
- "How many RowErg workouts have I done?"
- "Show me my average distance by equipment type"
- "What's my fastest pace on the SkiErg?"

In [ ]:
# Import required libraries
import asyncio
import pandas as pd
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from datetime import datetime
import sys
import os

print("✅ Libraries imported successfully!")
print(f"🐍 Using Python: {sys.executable}")
print(f"📦 Current working directory: {os.getcwd()}")
print("📁 Notebook is now in the same directory as dynamic_sql_generator.py")

The history saving thread hit an unexpected error (UnicodeEncodeError('utf-8', '# Import required libraries\nimport asyncio\nimport pandas as pd\nfrom IPython.display import display, HTML, clear_output\nimport ipywidgets as widgets\nfrom datetime import datetime\nimport sys\nimport os\n\nprint("✅ Libraries imported successfully!")\nprint(f"🐍 Using Python: {sys.executable}")\nprint(f"📦 Current working directory: {os.getcwd()}")\nprint("\udcc1 Notebook is now in the same directory as dynamic_sql_generator.py")', 347, 348, 'surrogates not allowed')).History will not be written to the database.


UnicodeEncodeError: 'utf-8' codec can't encode character '\udcc1' in position 7: surrogates not allowed

In [ ]:
# Import our dynamic SQL generator (now in the same directory)
try:
    from dynamic_sql_generator import SmartWorkoutQueryAssistant, DynamicSQLGenerator
    print("✅ Successfully imported SmartWorkoutQueryAssistant!")
    print("🧠 Dynamic SQL generation system loaded")
except ImportError as e:
    print(f"❌ Error importing: {e}")
    print("📍 Make sure dynamic_sql_generator.py exists in this directory")
    import os
    print(f"📁 Files in current directory: {[f for f in os.listdir('.') if f.endswith('.py')]}")

In [ ]:
# Initialize the query assistant
assistant = SmartWorkoutQueryAssistant(db_path="./c2_data.db")
sql_generator = DynamicSQLGenerator()

print("🚀 Workout Query Assistant initialized!")
print("🗄️ Connected to database: ./c2_data.db")
print("⚡ Ready to process natural language queries!")
print("🎯 Using the .venv virtual environment from this directory")

## 🧪 Test Individual Queries

Use the cell below to test individual natural language questions. You can modify the question and re-run the cell to see different results.

In [ ]:
# Test a single natural language query
async def test_single_query(question):
    """Test a single natural language question"""
    print(f"❓ Question: {question}")
    print("=" * 60)
    
    # Parse the question into intent
    intent = sql_generator.parse_natural_language(question)
    print(f"🧠 Parsed Intent:")
    print(f"   Action: {intent.action}")
    print(f"   Target: {intent.target}")
    print(f"   Filters: {intent.filters}")
    print(f"   Group By: {intent.groupby}")
    print(f"   Order By: {intent.orderby}")
    print(f"   Limit: {intent.limit}")
    print(f"   Aggregate: {intent.aggregate}")
    print()
    
    # Generate SQL
    sql = sql_generator.build_sql(intent)
    print(f"🔧 Generated SQL:")
    print(f"```sql\n{sql}\n```")
    print()
    
    # Execute query
    print("🎯 Results:")
    result = await assistant.ask_question(question)
    print(result)

# Test with a sample question (change this to test different questions)
question = "What was my longest workout?"
await test_single_query(question)

## 🎮 Interactive Query Interface

Use the interactive widget below to ask questions in real-time. Just type your question and click "Ask Question" to see the results!

In [ ]:
# Create interactive widgets
question_input = widgets.Text(
    value="What was my best 6k time?",
    placeholder="Ask a question about your workouts...",
    description="Question:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

ask_button = widgets.Button(
    description="Ask Question",
    button_style='primary',
    icon='search'
)

output_area = widgets.Output()

# Create some example buttons for quick testing
example_questions = [
    "What was my longest workout?",
    "What's my best 6k time?", 
    "How many RowErg workouts have I done?",
    "Show me my average distance by equipment type",
    "What's my fastest pace on the SkiErg?",
    "Show me my recent 10k workouts",
    "Count all my workouts by type"
]

example_buttons = []
for i, q in enumerate(example_questions):
    btn = widgets.Button(
        description=f"Example {i+1}",
        tooltip=q,
        layout=widgets.Layout(width='auto', margin='2px'),
        button_style='info'
    )
    example_buttons.append(btn)

# Function to handle question asking
async def ask_question_handler(b):
    with output_area:
        clear_output(wait=True)
        question = question_input.value.strip()
        if not question:
            print("❌ Please enter a question!")
            return
            
        print(f"🔄 Processing question: {question}")
        print("=" * 80)
        
        try:
            # Show the parsing and SQL generation
            intent = sql_generator.parse_natural_language(question)
            sql = sql_generator.build_sql(intent)
            
            print(f"🧠 Parsed Intent: {intent}")
            print(f"🔧 Generated SQL:\n{sql}")
            print("\n" + "="*50)
            
            # Execute the query
            result = await assistant.ask_question(question)
            print("🎯 Results:")
            print(result)
            
        except Exception as e:
            print(f"❌ Error: {str(e)}")

# Function to handle example button clicks
def example_click_handler(button_index):
    def handler(b):
        question_input.value = example_questions[button_index]
    return handler

# Connect button handlers
ask_button.on_click(lambda b: asyncio.create_task(ask_question_handler(b)))

for i, btn in enumerate(example_buttons):
    btn.on_click(example_click_handler(i))

# Display the interface
print("🎮 Interactive Query Interface")
print("Type your question below or click an example button:")

display(widgets.VBox([
    widgets.HTML("<h3>Ask a Question:</h3>"),
    question_input,
    ask_button,
    widgets.HTML("<h4>Or try these examples:</h4>"),
    widgets.HBox(example_buttons[:4]),
    widgets.HBox(example_buttons[4:]),
    widgets.HTML("<h4>Results:</h4>"),
    output_area
]))

## 📊 Query Analysis

The cells below help you understand how different types of questions are parsed and processed.

In [ ]:
# Analyze how different question types are parsed
test_questions = [
    "What was my longest workout?",
    "Show me my best 6k time",
    "How many RowErg workouts have I done?", 
    "What's my average distance by equipment type?",
    "What's my fastest pace on the SkiErg?",
    "Show me recent workouts",
    "What's my slowest 10k time?",
    "Count all workouts by type"
]

print("🔍 Natural Language Parsing Analysis")
print("=" * 80)

for i, question in enumerate(test_questions, 1):
    print(f"\n{i}. Question: {question}")
    intent = sql_generator.parse_natural_language(question)
    sql = sql_generator.build_sql(intent)
    
    print(f"   → Target: {intent.target}")
    print(f"   → Aggregate: {intent.aggregate}")
    print(f"   → Filters: {len(intent.filters)} filter(s)")
    print(f"   → Group By: {intent.groupby}")
    print(f"   → SQL Preview: {sql[:60]}{'...' if len(sql) > 60 else ''}")

## 🧠 Understanding the System

This section explains how the natural language processing works under the hood.

In [ ]:
# Show the system's understanding of different terms
print("🧠 System's Natural Language Understanding")
print("=" * 60)

print("\n🎯 Distance Terms:")
print("   Recognized:", sql_generator.distance_terms)

print("\n⏱️ Time Terms:")
print("   Recognized:", sql_generator.time_terms)

print("\n🏃 Pace Terms:")
print("   Recognized:", sql_generator.pace_terms)

print("\n🏋️ Equipment Terms:")
print("   Recognized:", sql_generator.equipment_terms)

print("\n📈 Aggregation Mappings:")
for word, sql_func in sql_generator.aggregations.items():
    print(f"   '{word}' → {sql_func}")

print("\n🗂️ Database Schema:")
print("   Table:", sql_generator.table_name)
print("   Columns:")
for friendly_name, db_column in sql_generator.columns.items():
    print(f"     {friendly_name} → {db_column}")

## 🎯 Custom Query Testing

Use this section to test your own custom questions and see exactly how they're processed.

In [ ]:
# Custom query tester - modify this cell to test your own questions
async def detailed_query_analysis(question):
    """Provides detailed analysis of how a question is processed"""
    print(f"🔍 Detailed Analysis for: '{question}'")
    print("=" * 80)
    
    # Step 1: Show original question
    print("1️⃣ Original Question:")
    print(f"   {question}")
    
    # Step 2: Show preprocessing
    processed = question.lower().strip()
    print(f"\n2️⃣ Preprocessed:")
    print(f"   {processed}")
    
    # Step 3: Parse intent
    print(f"\n3️⃣ Intent Parsing:")
    intent = sql_generator.parse_natural_language(question)
    print(f"   Action: {intent.action}")
    print(f"   Target: {intent.target}")
    print(f"   Aggregate: {intent.aggregate}")
    print(f"   Filters: {intent.filters}")
    print(f"   Group By: {intent.groupby}")
    print(f"   Order By: {intent.orderby}")
    print(f"   Limit: {intent.limit}")
    
    # Step 4: Show SQL generation
    print(f"\n4️⃣ SQL Generation:")
    sql = sql_generator.build_sql(intent)
    print(f"   {sql}")
    
    # Step 5: Execute and show results
    print(f"\n5️⃣ Execution Results:")
    try:
        result = await assistant.ask_question(question)
        print(result)
    except Exception as e:
        print(f"   ❌ Error: {e}")

# Test your custom question here (change the question to test different ones)
custom_question = "Show me my total distance for each equipment type"
await detailed_query_analysis(custom_question)

## 💡 Tips for Better Questions

Here are some tips for asking effective questions:

### ✅ What Works Well:
- **Specific distances**: "6k", "10000m", "5k"
- **Equipment types**: "RowErg", "SkiErg" 
- **Aggregations**: "best", "worst", "average", "total", "count"
- **Time references**: "recent", "latest"
- **Comparisons**: "by equipment type", "by type"

### 🔧 Question Patterns:
- **Finding records**: "What was my [best/worst/longest] [distance] [workout]?"
- **Counting**: "How many [equipment] workouts have I done?"
- **Averaging**: "What's my average [distance/pace] by [equipment type]?"
- **Recent data**: "Show me my recent [distance] workouts"

### 🚧 Current Limitations:
- Date ranges (e.g., "last month") are partially implemented
- Complex multi-condition queries need improvement
- Natural language pace comparisons could be enhanced

Feel free to experiment and see how the system handles different types of questions!